# LSTM implements

RNN/LSTM/GRU 에 대해 다양한 실험을 해 보는 것을 목표로 한다.

* feed-forward LSTM
    * https://notebooks.azure.com/hoyean-song/libraries/tensorflow-tutorial/html/LSTM%20breakdown.ipynb
* implements LSTM & GRU without cell wrapper
    * LSTMCell 같은거 사용하지 않고 tf 의 autogradient 기능만을 활용하여 구현
* implements LSTM & GRU using cell wrapper

여기까지를 일단 목표로!

## feed-forward LSTM

https://notebooks.azure.com/hoyean-song/libraries/tensorflow-tutorial/html/LSTM%20breakdown.ipynb

In [13]:
import tensorflow as tf
import numpy as np

### LSTM with numpy

아마 원래 식 대로면 h, x 에 대해서 W_hi, W_xi, W_hf, ... 등등 해서 수많은 weight 가 존재하는데 이걸 하나로 합칠 수 있는 듯 (물론 파라메터의 수는 그대로).  
일단 h 랑 x 랑 concat 해서 사용하면 절반으로 줄일 수 있고, i, f, c, o 에 대해서도 한번에 연산. 

다만 이 경우 원 식에 있는걸 그대로 재현이 안되는거 같은데... 일단 짜고 생각해보자.

In [14]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [15]:
# chk: what's this x, c, h ?
# x: input x
# h: hidden state of before cell (=> output of before cell)
# c: cell weights...?

x = np.array([[1., 1., 1.]])
c = 0.1 * np.array([[0, 1]])
h = 0.1 * np.array([[2, 3]])

num_units = 2
xh = np.concatenate([x, h], axis=1)

In [16]:
# every weights are set 0.5

W = np.ones([xh.shape[1], num_units*4]) * 0.5
b = np.ones([num_units*4]) * 0.5

In [17]:
f, i, o, g = np.split(np.matmul(xh, W) + b, 4, axis=1)

In [18]:
print f
print i
print o
print g

[[ 2.25  2.25]]
[[ 2.25  2.25]]
[[ 2.25  2.25]]
[[ 2.25  2.25]]


In [19]:
# forget_bias: https://github.com/pytorch/pytorch/issues/750
# forget gate 의 경우 이렇게 bias 를 높게 잡아주는것이 (0이 아니라 1로 잡아주는것이) 더 좋은 성능을 낸다나 봄.
forget_bias = np.ones_like(f)

new_c = sigmoid(f + forget_bias) * c + sigmoid(i) * np.tanh(g)
new_h = sigmoid(o) * np.tanh(new_c)

In [20]:
print new_c
print new_h

[[ 0.88477185  0.98103916]]
[[ 0.64121796  0.68166811]]


### LSTMCell in TF

In [21]:
from tensorflow.python.ops import variable_scope
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import variables
from tensorflow.contrib.rnn.python.ops import core_rnn_cell_impl
from tensorflow.contrib.rnn.python.ops import rnn_cell

In [22]:
tf.reset_default_graph()

with variable_scope.variable_scope("other", initializer=init_ops.constant_initializer(0.5)) as vs:
    x = array_ops.zeros([1, 3])  # Test BasicLSTMCell with input_size != num_units.
    c = array_ops.zeros([1, 2])
    h = array_ops.zeros([1, 2])
    state = core_rnn_cell_impl.LSTMStateTuple(c, h)
    cell = rnn_cell.LayerNormBasicLSTMCell(num_units=2, forget_bias=1.0, layer_norm=False)
    g, out_m = cell(x, state)

with tf.Session() as sess:
    sess.run(variables.global_variables_initializer())
        
    res = sess.run([g, out_m], {
      x: np.array([[1., 1., 1.]]),
      c: 0.1 * np.array([[0, 1]]),
      h: 0.1 * np.array([[2, 3]]),
    })

    print res[0]
    print res[1].c
    print res[1].h

#     expected_h = np.array([[ 0.64121795, 0.68166804]])
#     expected_c = np.array([[ 0.88477188, 0.98103917]])
    
#     assert np.sum(np.abs(res[1].h - expected_h)) <= 1e-6
#     assert np.sum(np.abs(res[1].c - expected_c)) <= 1e-6

    print np.sum(np.abs(res[1].h - new_h))
    print np.sum(np.abs(res[1].c - new_c))
    
    assert np.sum(np.abs(res[1].h - new_h)) <= 1e-6
    assert np.sum(np.abs(res[1].c - new_c)) <= 1e-6

[[ 0.64121789  0.68166804]]
[[ 0.88477182  0.98103911]]
[[ 0.64121789  0.68166804]]
1.32727097646e-07
7.66189002244e-08


In [23]:
x

<tf.Tensor 'other/zeros:0' shape=(1, 3) dtype=float32>

In [24]:
x.name

u'other/zeros:0'